# Импорт библиотек

In [1]:
import pandas as pd

# треккинг экспериментов
import wandb

# работа с S3
import boto3
import pickle
from io import BytesIO, StringIO

# предобработка данных
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split

from data_preproc.src.utils import preprocess_text5

# ML-модели
from sklearn.linear_model import LogisticRegression

# оценка моделей
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.model_selection import cross_validate

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirill.rubashevskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kirill.rubashevskiy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
BUCKET_NAME = "mlds23-authorship-identification"
DATA_DIR = "splitted_data/"
DATA_FILE_NAME = 'splitted_df.csv'
MODELS_DIR = 'models/'
RANDOM_STATE = 12345
RUN_NAME = 'kr-26-11-23-exp-1'

In [3]:
wandb.login()

wandb: Currently logged in as: kirill-rubashevskiy. Use `wandb login --relogin` to force relogin


True

In [4]:
# создание эксперимента
wandb.init(
    entity='mlds23_ai',
    project='authorship_identification',
    name=RUN_NAME,
    tags=['baseline']
)

wandb: Currently logged in as: kirill-rubashevskiy (mlds23_ai). Use `wandb login --relogin` to force relogin


In [5]:
wandb.config['random_state'] = 12345

# Загрузка данных с S3

In [6]:
session = boto3.session.Session()

In [7]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='REDACTED_KEY_ID_1',
    aws_secret_access_key='REDACTED_SECRET_KEY_1',
    region_name='ru-cental1'
)

In [8]:
csv_obj = s3.get_object(Bucket=BUCKET_NAME, Key=DATA_DIR + DATA_FILE_NAME)

In [9]:
data = pd.read_csv(StringIO(csv_obj['Body'].read().decode('utf-8')))

# Предобработка данных

In [10]:
# сохранение признаков и целевого признака в отдельные переменные
X = data['text'].apply(preprocess_text5)
y = data['target'].map(lambda x: int(x[-2:]))

In [11]:
# создание словаря для конвертации номеров классов в фамилии авторов
label2name = {
    0: 'А. Пушкин',
    1: 'Д. Мамин-Сибиряк',
    2: 'И. Тургенев',
    3: 'А. Чехов',
    4: 'Н. Гоголь',
    5: 'И. Бунин',
    6: 'А. Куприн',
    7: 'А. Платонов',
    8: 'В. Гаршин',
    9: 'Ф. Достоевский'
}

In [12]:
# разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [13]:
# разбиение обучающих данных на фолды
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [14]:
# создание объекта для TF-IDF
vectorizer = TfidfVectorizer()

In [15]:
# создание модели логистической регрессии
lr = LogisticRegression(penalty=None, random_state=RANDOM_STATE, solver='sag', max_iter=1000)

In [16]:
# создание пайплайна модели
model = Pipeline([
    ('preprocessing', vectorizer),
    ('classifier', lr)
])

In [17]:
# логирование гиперпараметров данных, предобработки и модели
wandb.config['dataset'] = DATA_FILE_NAME

wandb.config['preprocessing'] = {
    'lowercase': True,
    'remove_punct': True,
    'remove_html_tags': True,
    'remove_numbers': True,
    'lemmatization': False,
    'remove_stop_words': False,
}
wandb.config['embedding'] = {
    'embedding_type': 'TF-IDF'    
}

wandb.config['classifier'] = {
    'classifier_name': model['classifier'].__class__.__name__,
    'penalty': None,
    'solver': 'sag',
    'max_iter': 1000
}

wandb.config['evaluation'] = {
    'test_size': 0.25,
    'train_cv': 5
}

# Обучение и оценка модели на трейне

In [18]:
# расчет метрик на обучающей выборке при помощи кросс-валидации
cv_results = cross_validate(
    model,
    X_train, 
    y_train,
    scoring=['f1_micro', 'f1_macro', 'f1_weighted'],
    cv=skf
)

In [19]:
# логирование метрик
metrics_train = dict()

for metric in ['test_f1_micro', 'test_f1_macro', 'test_f1_weighted']:
    metrics_train[f'{metric[5:]}_mean'] = cv_results[metric].mean()
    metrics_train[f'{metric[5:]}_std'] = cv_results[metric].std()
    

wandb.log({
    'train': metrics_train
    })

# Оценка модели на тесте

In [20]:
# обучение модели на трейне и получение предсказаний на тесте
preds_test = model.fit(X_train, y_train).predict(X_test)

In [21]:
# логирование метрик на тесте
metrics_test = dict()

for average in ['micro', 'weighted', 'macro']:
    metrics_test[f'f1_{average}'] = f1_score(y_test, preds_test, average=average)
    
wandb.log({
    'test': metrics_test
})

In [22]:
# логирование метрик на тесте с разбивкой по классу
precision_recall_f1_test_df = pd.DataFrame(
    precision_recall_fscore_support(y_test, preds_test),
    columns=list(label2name.values()),
    index = ['precision_test', 'recall_test', 'f1_test', 'support']).T.drop(columns=['support']).reset_index()

precision_recall_f1_test_df.rename(columns={'index': 'author'}, inplace=True)

wandb.log({
    'precision_recall_f1_table_test': wandb.Table(dataframe=precision_recall_f1_test_df)   
})

In [23]:
# логирование confusion matrix на тесте
wandb.log({
    'conf_mat_test': wandb.plot.confusion_matrix(probs=None, y_true=y_test.tolist(), preds=preds_test, class_names=list(label2name.values()))   
})


# Сохранение модели

In [24]:
# сохранение модели на S3 через буффер (без создания локальной копии
pickle_buffer = BytesIO()
pickle_byte_obj = pickle.dump(model, pickle_buffer)

s3.put_object(Body=pickle_buffer.getvalue(), 
              Bucket=BUCKET_NAME, 
              Key=f'{MODELS_DIR}{RUN_NAME}_pipeline.pkl')

{'ResponseMetadata': {'RequestId': '60e6a8f50e9c3ed0',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Mon, 27 Nov 2023 09:18:22 GMT',
   'content-type': 'application/octet-stream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'etag': '"25dd6dbc3641ddb44b710c752063da97"',
   'x-amz-request-id': '60e6a8f50e9c3ed0'},
  'RetryAttempts': 0},
 'ETag': '"25dd6dbc3641ddb44b710c752063da97"'}

In [25]:
# завершение эксперимента
wandb.finish()